### Toy Lindblad identification as POP solved with Moment SDP

In [1]:
using LinearAlgebra, JuMP, MosekTools
using Optim

Consider SDP optimization problem of the form:
\begin{align} 
&\operatorname{minimize}{ \left( \mathbf{trace}{(C X)} \right) }\\
&\operatorname{subject to} { y_i := \left( \mathbf{trace}{(A_i X)} \right) = b_i}, \quad i = 1... m, \quad
X \succcurlyeq 0
\end{align}

where $X\in\mathbf{S}^{n}$ is the decision variable, and each of the $A_{i}$ matrices and $C$ are also in $\mathbf{S}^{n}$  By the notation $\mathbf{S}^{n}$, we denote the set of all symmetric $n \times n$ matrices.

It could be solved with the following function:

In [2]:
function solve_SDP(A, b, C; quiet = false, term_tolerance = 1e-7)
    
    start_time = time()
    
    n = size(A)[1] # = size(C)[1]
    m = length(b)

    model = Model(optimizer_with_attributes(Mosek.Optimizer, "QUIET" => quiet, "INTPNT_CO_TOL_DFEAS" => term_tolerance))

    #set_silent(model)

    @variable(model, X[1:n, 1:n], PSD)

    @objective(model, Min, tr(C * X));
    
    @constraint(model, y[j=1:m], tr(A[:, (j - 1) * n + 1:j * n]* X) == b[j])

    optimize!(model)

    status = JuMP.termination_status(model)
    X_sol = JuMP.value.(X)
    obj_value = JuMP.objective_value(model)
    dual_sol = dual.(y)
    run_time = time() - start_time

    return model, status, run_time, X_sol, dual_sol, obj_value
    
end

solve_SDP (generic function with 1 method)

Within the Markovian approximation when the trajectory of the system is determined by its current state and do not depend on the previous history of its evolution we can use Lindblad master equation:

$\mathcal{L}\left[\rho\right] = - i [H, \rho]+\sum_{\ell=1}^{s-1}\left[A_\ell
\rho A_\ell^\dagger - \frac{1}{2}\left\{ A_\ell^\dagger A_\ell, \rho \right\} \right]$

Consider simplified Lindblad type equation:

$
    \frac{d\rho}{dt} = - i[H, \rho]+\gamma\left[A \rho A^\dagger - \frac{1}{2}\left\{ A^\dagger A, \rho \right\} \right],
$

where
$A = \begin{pmatrix} 0 & a \\ 0 & 0
   \end{pmatrix}$ and
$ H =  \begin{pmatrix} \omega & 0 \\ 0 & 0
    \end{pmatrix}$

To identify parameters of Lindblad equation we need to find optimum values of  angular frequency $\omega$ and decay rate $\gamma$.


We can formulate the following minimization problem using Pade approximation and just the initial and final state of the evolution (we do it here just for illustrative purposes as in practice we can do it only if $\Delta t$ is small):

$\min_{\gamma, H}{{\left\|{ \rho_f - \rho_0 - \Delta t {\mathcal{L}} \left[\frac{ \rho_f + \rho_0}{2} \right]} \right\|}_F^2}$

where  $\mathcal{L}$ is the Lindbladian operator:

$\mathcal{L} = - i[H, \rho]+\gamma\left[A \rho A^\dagger - \frac{1}{2}\left\{ A^\dagger A, \rho \right\} \right]$

Consider Hamiltonian $H$ and dissipator  $A$  of a very simple form


In [3]:
using DynamicPolynomials
using LinearAlgebra

@polyvar ω a

H = [ω 0
     0 0]
A = [0 a
     0 0]

2×2 Matrix{Term{true, Int64}}:
 0  a
 0  0

Now consider transfer:
$ |y \rangle  = \frac{|0 \rangle + i|1\rangle}{\sqrt{2}} \rightarrow |\psi \rangle  = \sqrt{\frac{2}{3}}|0\rangle + \frac{1}{\sqrt{3}} |1 \rangle   $

In [4]:
ρ₁ = [1   im
      -im  1]/2

ρ₂ = [ 2 √2 
      √2  1]/3

ρ = (ρ₁ + ρ₂)/2

L = -im * (H * ρ - ρ * H) + (A * ρ * A' - (A' * A  * ρ + ρ * A' * A) / 2)
M = (ρ₂ - ρ₁) - L

obj = tr(M * M')

(0.40624999999999994 + 0.0im)a⁴ + (0.23611111111111113 + 0.0im)ω² + (-0.30555555555555547 + 0.0im)a² + (-0.9428090415820635 + 0.0im)ω + (1.0 + 0.0im)

In [5]:
import Base.real

function real(p::AbstractPolynomial)
    sum(Base.real(coef) * mon for (coef, mon) in zip(coefficients(p), monomials(p))) #if ~isapproxzero(abs(coef)))
end

obj = real(obj)

0.40624999999999994a⁴ + 0.23611111111111113ω² - 0.30555555555555547a² - 0.9428090415820635ω + 1.0

we have objective function:
$
L = {\left\|{ \rho_2 - \rho_1 - \Delta t {\mathcal{L}} \left[\frac{ \rho_1 + \rho_2}{2} \right]} \right\|}_F^2 =
1 - \frac{11 }{36}\Delta t a^2 + \frac{13 }{32}\Delta t^2 a^4 - \frac{2 \sqrt{2} }{3}\Delta t\omega + \frac{17 }{72}\Delta t^2\omega^2
$,

where 

In [6]:
17/72, 13/32, -2√2/3, -11/36, 1

(0.2361111111111111, 0.40625, -0.9428090415820635, -0.3055555555555556, 1)

Introducing $x_1 = \Delta t \omega$ and $ x_2 = \Delta t a$ we can rewrite the objective as a following polynomial optimization problem:

$
p(x) = 1 - \frac{2 \sqrt2}{3}x_1 -\frac{11}{36} x_2^2  + \frac{17}{72}x_1^4 + 0x_1x_2 + \frac{13}{32}x_2^4
$

$\min_{x \in \mathbb{R}^2 } p(x)$

In [7]:
function p(x)
    1 -(2√2/3)*x[1] -(11/36)*x[2]^2 + (17/72)*x[1]^2 + (13/32)*x[2]^4
end

p (generic function with 1 method)

For such a simple polynomial minimizers could be found analiticaly
or by standard numerical methods.

Analitical solution:

In [8]:
@show x₁ᵃⁿᵃˡⁱᵗ = 24√2/17 # ω
@show x₂ᵃⁿᵃˡⁱᵗ = 2*sqrt(11/13)/3 # a = 2*sqrt(11/13)/3, γ = 44/117
p([x₁ᵃⁿᵃˡⁱᵗ, x₂ᵃⁿᵃˡⁱᵗ])

x₁ᵃⁿᵃˡⁱᵗ = (24 * √2) / 17 = 1.9965367939384873
x₂ᵃⁿᵃˡⁱᵗ = (2 * sqrt(11 / 13)) / 3 = 0.6132441406718666


0.0013686386235402595

Numerical solution with BFGS:

In [9]:
result = optimize(obj, zeros(2), BFGS())
@show x₁ᵇᶠᵍˢ, x₂ᵇᶠᵍˢ = Optim.minimizer(result)
p([x₁ᵇᶠᵍˢ, x₂ᵇᶠᵍˢ])

(x₁ᵇᶠᵍˢ, x₂ᵇᶠᵍˢ) = Optim.minimizer(result) = [1.9965367939244139, 0.0]


0.05882352941176461

#### We can also try to solve our polynomial optimization problem as SDP in the space of moments

We can rewrite polynomial $p(x)$ as linear combination of monomials
$
p(x) = \sum_{\alpha \leq d} p_\alpha x^\alpha,   
$

where $x = \left( x_1, x_2 \right)^T$ and $\alpha = (\alpha_1, \alpha_2), \quad \alpha_i \in \{0 \dots d\}$, as in our case $x \in \mathbb{R}^2$ thus $d=2$ and for $\alpha$ we have all combinations of degrees $\alpha_i = 0,1,2$.

To solve our problem as SDP in the space of moments we move in the following steps:

#### 1. Infinite-dimensional LP
Reformulate the Nonlinear Optimization problem in terms of Probability Distributions (measures) to obtain the Infinite-dimensional Linear Programming problem in infinite-dimensional space of measures.

$\min_{\mu \in \mathcal{M}(\mathcal{X}} {\operatorname{E} [p(x)] = \sum_\alpha^\infty p_\alpha y_\alpha}$



#### 2. Infinite-dimensional SDP 

Reformulate the infinite-dimensional Problem of Step 1 in terms of Moments (higher order statistics) to obtain an infinite-dimensional Semi-Definite Programming problem.

$\min_{y \in \mathcal{Y}}{\operatorname{E} [p(x)] = \sum_\alpha^\infty p_\alpha y_\alpha}{}$
 
$\operatorname{s.t.}
 M_1(y), M_2(y), M_3(y) \dots \succcurlyeq 0$,
 
where $y_\alpha$ are moments defined as follows:
$
    y_\alpha := \int_\mathcal{X} x^\alpha \mu(dx),
$

#### 3.Finite SDP 

Truncate matrices of infinite-dimensional Semi-Definite Programming problem of Step 2 (use truncated moments) to obtain finite-dimensional Semi-Definite Programming problem.

For example, for two-dimensional $x \in \mathbb{R}^2 \mapsto p(x)$ we can write the first relaxation of it:

$\min_{y \in \mathbb{R}^6 } {\sum_{\alpha_1, \alpha_2 = 0}^{2} p_\alpha y_\alpha}$

$\operatorname{s.t.}{y_{00} = 1 }$

$  
 M_1(y) =\begin{pmatrix} 
y_{00} & y_{10} & y_{01}\\
y_{10} & y_{20} & y_{11}\\
y_{01} & y_{11} & y_{02} 
\end{pmatrix} \succcurlyeq 0
$

#### In our case:

Having polynomial optimization problem:

$
p(x) = 1 - \frac{2 \sqrt2}{3}x_1 -\frac{11}{36} x_2^2  + \frac{17}{72}x_1^2 + 0x_1x_2 + \frac{13}{32}x_2^4
$

$
 = 1 x_1^0 x_2^0 -\frac{2 \sqrt2}{3} x_1^1 x_2^0 - \frac{11}{36} x_1^0 x_2^2 + \frac{17}{72}x_1^2 x_2^0 + 0x_1^1x_2^2 + \frac{13}{32}x_1^0x_2^4
$

$\min_{x \in \mathbb{R}^2 } p(x)$

we can write the first and second linear matrix inequality (LMI) relaxation for it:
    
$\min_{y \in \mathbb{R}^6 }
    1 y_{00} - \frac{2 \sqrt2}{3} y_{10} - \frac{11}{36} y_{02} + \frac{17}{72}y_{20} + \frac{13}{32}y_{04}$
    
$\operatorname{s.t.}{y_{00} = 1, M_1(y)\succcurlyeq 0}$

where

$M_1(y) =\begin{pmatrix} 
y_{00} & y_{10} & y_{01}\\
y_{10} & y_{20} & y_{11}\\
y_{01} & y_{11} & y_{02}
\end{pmatrix}$

$M_2(y) =\begin{pmatrix} 
    y_{00} & y_{10} & y_{01} & y_{20} & y_{11} & y_{02}\\
    y_{10} & y_{20} & y_{11} & y_{30} & y_{21} & y_{12}\\
    y_{01} & y_{11} & y_{02} & y_{21} & y_{12} & y_{03}\\
    y_{20} & y_{30} & y_{21} & y_{40} & y_{31} & y_{22}\\
    y_{11} & y_{21} & y_{12} & y_{31} & y_{22} & y_{13}\\
    y_{02} & y_{12} & y_{03} & y_{22} & y_{13} & y_{04}\\
    \end{pmatrix}$

This is SDP problem, 

$
\operatorname{minimize}{ \left( \mathbf{trace}{(C X)} \right) }\\
\operatorname{subject to} { y_i := \left( \mathbf{trace}{(A_i X)} \right) = b_i}, \quad i = 1... m, 
X \succcurlyeq 0
$

where:

$C =\begin{pmatrix} 
1 & -\frac{\sqrt2}{3} & 0 & \frac{17}{72}/3 & 0 &-\frac{11}{36}/3 \\
-\frac{\sqrt2}{3}  &  \frac{17}{72}/3  & 0  & 0 & 0 & 0\\
0 & 0 &  -\frac{11}{36}/3  & 0 & 0 & 0\\
\frac{17}{72}/3 & 0 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 & 0 \\
-\frac{11}{36}/3 & 0 & 0 & 0 & 0 & \frac{13}{32}\\
\end{pmatrix}$

target variables:
$X =\begin{pmatrix} 
    y_{00} & y_{10} & y_{01} & y_{20} & y_{11} & y_{02}\\
    y_{10} & y_{20} & y_{11} & y_{30} & y_{21} & y_{12}\\
    y_{01} & y_{11} & y_{02} & y_{21} & y_{12} & y_{03}\\
    y_{20} & y_{30} & y_{21} & y_{40} & y_{31} & y_{22}\\
    y_{11} & y_{21} & y_{12} & y_{31} & y_{22} & y_{13}\\
    y_{02} & y_{12} & y_{03} & y_{22} & y_{13} & y_{04}\\
    \end{pmatrix} \succcurlyeq 0$

and as $y_{00}=1$:

$A_1 =\begin{pmatrix} 
1 & 0 & 0 & 0 & 0 & 0\\
0 & 0 & 0 & 0 & 0 & 0\\
0 & 0 & 0 & 0 & 0 & 0\\
0 & 0 & 0 & 0 & 0 & 0\\
0 & 0 & 0 & 0 & 0 & 0\\
0 & 0 & 0 & 0 & 0 & 0\\
\end{pmatrix}, \quad b_1 = 1$


In [10]:
using DynamicPolynomials

@polyvar y[0:4, 0:4]

@polyvar x[1:2]

#C = [1           -√2/3        0      (17/72)/3    0     -(11/36)/3 
#     -√2/3       (17/72)/3    0       0           0      0 
#     0             0      -(11/36)/3  0           0      0         
#     (17/72)/3     0          0       0           0      0 
#     0             0          0       0           0      0 
#    -(11/36)/3     0          0       0           0      13/32 ]

C = [1           -√2/3        0      -(11/36)/3    0     (17/72)/3 
     -√2/3       -(11/36)/3   0       0           0      0 
     0             0      (17/72)/3   0           0      0         
     -(11/36)/3    0          0       0           0      0 
     0             0          0       0           0      0 
    (17/72)/3      0          0       0           0      13/32 ]

6×6 Matrix{Float64}:
  1.0        -0.471405  0.0        -0.101852  0.0  0.0787037
 -0.471405   -0.101852  0.0         0.0       0.0  0.0
  0.0         0.0       0.0787037   0.0       0.0  0.0
 -0.101852    0.0       0.0         0.0       0.0  0.0
  0.0         0.0       0.0         0.0       0.0  0.0
  0.0787037   0.0       0.0         0.0       0.0  0.40625

In [11]:
Xˢʸᵐᵇ = [ y[1,1] y[2,1] y[1,2] y[3,1] y[2,2] y[1,3]
          y[2,1] y[3,1] y[2,2] y[4,1] y[3,2] y[2,3]
          y[1,2] y[2,2] y[1,3] y[3,2] y[2,3] y[1,4]
          y[3,1] y[4,1] y[3,2] y[5,1] y[4,2] y[3,3]
          y[2,2] y[3,2] y[2,3] y[4,2] y[3,3] y[2,4]
          y[1,3] y[2,3] y[1,4] y[3,3] y[2,4] y[1,5] ]

6×6 Matrix{PolyVar{true}}:
 y₀₋₀  y₁₋₀  y₀₋₁  y₂₋₀  y₁₋₁  y₀₋₂
 y₁₋₀  y₂₋₀  y₁₋₁  y₃₋₀  y₂₋₁  y₁₋₂
 y₀₋₁  y₁₋₁  y₀₋₂  y₂₋₁  y₁₋₂  y₀₋₃
 y₂₋₀  y₃₋₀  y₂₋₁  y₄₋₀  y₃₋₁  y₂₋₂
 y₁₋₁  y₂₋₁  y₁₋₂  y₃₋₁  y₂₋₂  y₁₋₃
 y₀₋₂  y₁₋₂  y₀₋₃  y₂₋₂  y₁₋₃  y₀₋₄

We can check the coefficients are the same:

$\min_{y \in \mathbb{R}^6 }
    1 y_{00} - \frac{2 \sqrt2}{3} y_{10} - \frac{11}{36} y_{02} + \frac{17}{72}y_{20}  + \frac{13}{32}y_{04}$

In [12]:
1, -2√2/3, 17/72, -11/36, 13/32

(1, -0.9428090415820635, 0.2361111111111111, -0.3055555555555556, 0.40625)

In [13]:
tr(C * Xˢʸᵐᵇ)

y₀₋₀ - 0.9428090415820635y₁₋₀ - 0.3055555555555556y₂₋₀ + 0.2361111111111111y₀₋₂ + 0.40625y₀₋₄

In [14]:
p(x)

0.40625x₂⁴ + 0.2361111111111111x₁² - 0.3055555555555556x₂² - 0.9428090415820635x₁ + 1.0

Additional constaint for $y_{00}$:

In [24]:
A = zeros(6,6)
A[1,1] = 1
b = 1
A

6×6 Matrix{Float64}:
 1.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0

Now we can solve SDP

In [25]:
model, status, run_time, X_sol, dual_sol, obj_value = solve_SDP(A, b, C)

Problem
  Name                   :                 
  Objective sense        : min             
  Type                   : CONIC (conic optimization problem)
  Constraints            : 1               
  Cones                  : 0               
  Scalar variables       : 0               
  Matrix variables       : 1               
  Integer variables      : 0               

Optimizer started.
Presolve started.
Linear dependency checker started.
Linear dependency checker terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator started.
Freed constraints in eliminator : 0
Eliminator terminated.
Eliminator - tries                  : 2                 time                   : 0.00            
Lin. dep.  - tries                  : 1                 time                   : 0.00            
Lin. dep.  - number                 : 0               
Presolve terminated. Time: 0.00    
Problem
  Name                   :                 
  Objective se

(A JuMP Model
Minimization problem with:
Variables: 21
Objective function type: AffExpr
`AffExpr`-in-`MathOptInterface.EqualTo{Float64}`: 1 constraint
`Vector{VariableRef}`-in-`MathOptInterface.PositiveSemidefiniteConeTriangle`: 1 constraint
Model mode: AUTOMATIC
CachingOptimizer state: ATTACHED_OPTIMIZER
Solver name: Mosek
Names registered in the model: X, y, MathOptInterface.DUAL_INFEASIBLE, 0.003999948501586914, [2.6920552142381307e-13 1.486206698422724e-8 … 8.838594577877473e-36 1.9441866582830225e-8; 1.486206698422724e-8 4.073933863454154 … -7.293934520174133e-30 -0.0026839472594665558; … ; 8.838594577877473e-36 -7.293934520174133e-30 … 1.0 6.173223074884957e-30; 1.9441866582830225e-8 -0.0026839472594665558 … 6.173223074884957e-30 0.18307299601297775], [-0.21076384705595583], -0.29575000158187165)

In [26]:
X_sol

6×6 Matrix{Float64}:
  2.69206e-13   1.48621e-8   4.09016e-22  …   8.83859e-36   1.94419e-8
  1.48621e-8    4.07393      5.49377e-16     -7.29393e-30  -0.00268395
  4.09016e-22   5.49377e-16  0.569405         1.44416e-16   1.50329e-16
 -2.85381e-7    0.0252876    1.5916e-16       7.35902e-30   0.0101319
  8.83859e-36  -7.29393e-30  1.44416e-16      1.0           6.17322e-30
  1.94419e-8   -0.00268395   1.50329e-16  …   6.17322e-30   0.183073

#### Compare methods

In [29]:
x₁ˢᵈᵖ = sqrt(X_sol[2,2]) # ω = √x₁²

2.0183988365667855

In [30]:
x₂ˢᵈᵖ = sqrt(X_sol[3,3]) # a = √x₂²

#x₁ˢᵈᵖ, x₂ˢᵈᵖ = X_sol[1,2], X_sol[1,3]

0.7545890608540893

In [31]:
x₁ᵇᶠᵍˢ, x₂ᵇᶠᵍˢ

(1.9965367939244139, 0.0)

In [32]:
x₁ᵃⁿᵃˡⁱᵗ, x₂ᵃⁿᵃˡⁱᵗ

(1.9965367939384873, 0.6132441406718666)

In [33]:
[x₁ᵇᶠᵍˢ, x₂ᵇᶠᵍˢ] - [x₁ᵃⁿᵃˡⁱᵗ, x₂ᵃⁿᵃˡⁱᵗ]

2-element Vector{Float64}:
 -1.407340910475341e-11
 -0.6132441406718666

The precission of SDP is around $10^{-6}$:

In [34]:
[x₁ˢᵈᵖ, x₂ˢᵈᵖ] - [x₁ᵃⁿᵃˡⁱᵗ, x₂ᵃⁿᵃˡⁱᵗ]

2-element Vector{Float64}:
 -1.9965367790764204
  0.14134492018222267

In [35]:
result = optimize(obj, [x₁ˢᵈᵖ, x₂ˢᵈᵖ], BFGS())
@show x₁ˢᵈᵖᵇᶠᵍˢ, x₂ˢᵈᵖᵇᶠᵍˢ = Optim.minimizer(result)
p([x₁ˢᵈᵖᵇᶠᵍˢ, x₂ˢᵈᵖᵇᶠᵍˢ])

(x₁ˢᵈᵖᵇᶠᵍˢ, x₂ˢᵈᵖᵇᶠᵍˢ) = Optim.minimizer(result) = [1.9965367953352218, 0.6132441399986011]


0.0013686386235404469

In [36]:
using TSSOS
opt,sol,data = tssos_first(obj, variables(obj), QUIET=true, solution=true);
previous_sol = sol

************************TSSOS************************
TSSOS is launching...
optimum = 0.001368660301030318

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

Global optimality certified!


2-element Vector{Float64}:
 1.9965367939384873
 0.6132441406933407